# Оглавление
1. [Открытие данных](#open_data)
2. [Предобработка данных](#preprocessing)
    * [Обработка пропусков](#null)
    * [Замена типа данных](#astype)
    * [Обработка дубликатов](#duplicates)
    * [Лемматизация](#lemmatize)
    * [Категоризация](#categorize)
3. [Ответы на вопросы исследования](#answer_the_questions)
4. [Общий вывод](#recap)
5. [Чек-лист готовности проекта](#check_list)

# Исследование надёжности заёмщиков

**Заказчик исследования:** кредитный отдел банка.

**Задача:** определить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

**Цели:**
1. Импортировать данные и ознакомится с ними;
2. Провести предобработку данных;
3. Проверить гипотезу влияния семейного положения и наличия детей на погашение клиентом кредита в срок.

**Входные данные от заказчика:** статистика о платёжеспособности клиентов банка.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию
<a id="open_data"></a>

In [1]:
#В начале работы над проектом загрузили необходимые библиотеки
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
from IPython.display import display
import numpy as np

In [2]:
#Прочитаем исходный файл с данными
credit_data = pd.read_csv('/datasets/data.csv')

#Изучим общую информацию о данных, оценим типы столбцов и количество пропусков
credit_data.info()

#Визуально ознакомимся со структурой данных
display(credit_data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

Обращает на себя внимание артефакт – одинаковое число пропусков в столбцах 'days_employed' и 'total_income'. Ещё предстоит работа со столбцом 'days_employed', т.к., согласно информации из описания данных, в нём содержится общий трудовой стаж в днях, но значения там отрицательные и имеются явные ошибки на примере данных из 4 строки.

## Шаг 2. Предобработка данных
<a id="preprocessing"></a>

### Обработка пропусков
<a id="null"></a>

Определим точное количество строк с пропуском общего трудового стажа в днях и убедимся, что строки с отсутствующим значением 'days_employed' это те же строки, где отсутсвует 'total_income' (ежемесячный доход).

In [3]:
print(credit_data[credit_data['days_employed'].isnull()].count())

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64


Около 10% строк без указания 'days_employed' и 'total_income'. Просто удалить их было бы неправильно, ценных данных много. Оценим визуально, есть ли зависимость пропусков от других столбцов, можно ли попытаться восстановить данные.

In [4]:
display(credit_data[credit_data['days_employed'].isnull()].head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Данные в остальных столбцах имеют обыкновенный разброс. Зависимость пропусков от количества детей, уровня образования, семейного статуса, типа занятости, цели получения кредита или других показателей не наблюдается. Это значит, что выборка с пропусками примерно соответствует данным из остальной таблицы и пропуски можно заменить средним или медианным значением. Поскольку искать зависимость невозврата кредитов от стажа по условию проекта не придётся, то разницы между средним значением и медианой в данном случае нет. Но в выгрузке есть явная ошибка – значения в столбце 'days_employed' либо отрицательные, либо >300 тыс., что в днях  стремится к 1000 лет трудового стажа. Если бы данные из этого столбца были чувствительны для исследования, следовало бы обратиться за новой более точной выгрузкой.

In [5]:
credit_data['days_employed'] = credit_data['days_employed'].fillna(credit_data['days_employed'].mean())

#В случае с уровлнем дохода медиана точнее показывает средний уровень, поэтому применим её
credit_data['total_income'] = credit_data['total_income'].fillna(credit_data['total_income'].median())

#запустим метод .info(), чтобы убедится, что пропусков больше нет
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В 2174 строках таблицы пропущены данные о стаже и доходе. Возможно, данные "подтягивались" из разных источников и в одном из них этих столбцов небыло вовсе. Либо выгружали за несколько раз и в одну из "волн" произошла ошибка выгрузки.
Помимо пропусков, в столбце 'days_employed' есть логическая ошибка: дней стажа либо больше 800 лет, либо стаж отрицательный. Поскольку в данном исследовании искать зависимость возврата кредита от стажа задачи нет, то это не является проблемой. В противном случае, мы бы пошли за новой выгрузкой.
Пропуски в столбце с уровнем дохода мы заменили на медианное значение, так как в данном случае она точнее передаёт среднюю "температуру по палате".

### Замена типа данных
<a id="astype"></a>

In [6]:
credit_data['total_income'] = credit_data['total_income'].astype('int')
credit_data['days_employed'] = credit_data['days_employed'].astype('int')

Изменили вещественный тип данных на целочисленный для простоты дальнейших расчётов. Потерянные отброшенной точкой копейки в столбце дохода не повлияют на точность дальнейших выводов. Стаж заменили на целые числа, чтобы дробная часть не перегружала таблицу. Данный столбец для дальнейших выводов нам не интересен.

In [7]:
#уровняем Загланые буквы
credit_data['education'] = credit_data['education'].str.lower()
credit_data['family_status'] = credit_data['family_status'].str.lower()
credit_data['gender'] = credit_data['gender'].str.lower()
credit_data['income_type'] = credit_data['income_type'].str.lower()
credit_data['purpose'] = credit_data['purpose'].str.lower()

Уровняли заглавные буквы для простоты работы с дубликатами и для замены столбцов с ограниченным вариантом значений с типа 'object' на тип 'category', так мы сэкономим немного памяти.

In [8]:
credit_data['education'] = credit_data['education'].astype('category')
credit_data['family_status'] = credit_data['family_status'].astype('category')
credit_data['income_type'] = credit_data['income_type'].astype('category')
credit_data['gender'] = credit_data['gender'].astype('category')

Наличие и отсутствие задолженности по возврату кредитов логично было бы заменить на булевы значения.

In [9]:
credit_data['debt'] = credit_data['debt'].astype('bool')

В столбцах 'children', 'dob_years', 'education_id' и 'family_status_id' нет необходимости держать запас символов такой как в int64, поэтому сэкономим на них память, заменив тип на int8.

In [10]:
credit_data['children'] = credit_data['children'].astype('int8')
credit_data['dob_years'] = credit_data['dob_years'].astype('int8')
credit_data['education_id'] = credit_data['education_id'].astype('int8')
credit_data['family_status_id'] = credit_data['family_status_id'].astype('int8')

credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int8
days_employed       21525 non-null int64
dob_years           21525 non-null int8
education           21525 non-null category
education_id        21525 non-null int8
family_status       21525 non-null category
family_status_id    21525 non-null int8
gender              21525 non-null category
income_type         21525 non-null category
debt                21525 non-null bool
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: bool(1), category(4), int64(2), int8(4), object(1)
memory usage: 694.7+ KB


In [11]:
save_memory = 694.7 / 2048
print('Нам удалось сократить использование памяти до {:.0%} от изначальной'.format(save_memory))

Нам удалось сократить использование памяти до 34% от изначальной


**Вывод**

При помощи метода .astype() мы поменяли вещественный тип данных на целочисленный в столбцах с уровнем дохода и стажем. Факт наличия задолженности теперь хранится в таблице как булевы значения. Столбцы с ограниченным вариантом заполнения такие как уровень образования, семейное положение, пол и тип занятости были заменены на тип 'category'. Информация о количестве детей, возрасте, идентификаторы уровня образования и семейного положения не требовали того набора разнообразия в значениях, которое предоставляет тип int64, поэтому они были заменены на int8 без потери данных.
Всё это позволило сократить использование памяти при работе с датафреймом на 66%.

### Обработка дубликатов
<a id="duplicates"></a>

In [12]:
#Узнаем количество дубликатов
print(credit_data.duplicated().sum())

#71 дубль – немного для таблицы из 21,5 тыс. строк. Удалим их
credit_data.drop_duplicates().reset_index(drop = True)

71


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,f,сотрудник,False,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,f,сотрудник,False,112080,приобретение автомобиля
2,0,-5623,33,среднее,1,женат / замужем,0,m,сотрудник,False,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,m,сотрудник,False,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,False,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529,43,среднее,1,гражданский брак,1,f,компаньон,False,224791,операции с жильем
21450,0,343937,67,среднее,1,женат / замужем,0,f,пенсионер,False,155999,сделка с автомобилем
21451,1,-2113,38,среднее,1,гражданский брак,1,m,сотрудник,True,89672,недвижимость
21452,3,-3112,38,среднее,1,женат / замужем,0,m,сотрудник,True,244093,на покупку своего автомобиля


**Вывод**

В датафрейме обнаружено 71 дубликат. Обнаружили их методом для обнаружения дубликатов и их подсчёта .duplicated().sum(), выбрали его, так как убедились в его работоспособности при работе на тренажёре.
Статистически, дубликаты могли возникнуть после присвоения одинаковых значений пропускам в столбцах стажа и дохода, либо потенциальные заёмщики подавали по несколько заявок на кредит и данные дублировались.

### Лемматизация
<a id="lemmatize"></a>

In [13]:
#Оценим разнообразие целей получения кредитов
print(credit_data['purpose'].value_counts())

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Свадьба, жильё/недвижимость/ремонт, образование, автомобиль. Если бы составители анкеты-опросника потрудились с чекбоксами, а не оставили открытое поле, можно было бы сократить время заполнения 21,5 тыс. анкет и обойтись без лемматизации. Погнали

In [14]:
def lem_ka(row):
    lem_code = m.lemmatize(row['purpose'])
    if 'автомобиль' in lem_code:
        return 'автомобиль'
    if 'свадьба' in lem_code:
        return 'свадьба'
    if 'жилье' in lem_code:
        return 'недвижимость'
    if 'недвижимость' in lem_code:
        return 'недвижимость'
    if 'образование' in lem_code:
        return 'образование'

#Добавим столбец с лемматизированным поводом получения кредита
credit_data['lem_code'] = credit_data.apply(lem_ka, axis = 1)

**Вывод**

Визуально оценили ограниченный список поводов для получения кредита из выборки. Сгруппировали основные потребности заёмщиков до 4х категорий и добавили лемматизированный столбец для дальнейшего исследования.

### Категоризация данных
<a id="categorize"></a>

In [15]:
#Займёмся словарями
education_dict = credit_data[['education', 'education_id']].drop_duplicates().reset_index(drop = True)
family_status_dict = credit_data[['family_status', 'family_status_id']].drop_duplicates().reset_index(drop = True)

#Распечатаем словари для простоты дальнейшей работы
display(education_dict)
display(family_status_dict)

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


In [16]:
#Категоризируем клиентов по уровню дохода
def income_lvl(row):
    if row['total_income'] < 125000:
        return 'низкий'
    if row['total_income'] <= 170000:
        return 'средний'
    if row['total_income'] > 170000:
        return 'высокий'

#Разыграем функцию
credit_data['category_lvl_income'] = credit_data.apply(income_lvl, axis = 1)

Методом подбора определили примерные рубежи в 125 и 170 тыс. руб., которые разделяют уровни дохода респондентов на 3 ±равные части. После чего написали функцию, которая присваивает категорию уровня дохода для простоты дальнейшего исследования.

In [17]:
print(credit_data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Устраним атрефакты в столбце 'children': методом .value_counts() узнаём, что присутствуют нелогичные и выдающиеся показатели -1 (47 строк) и 20 (76 строк). Удалять строки и терять ценные данные не хотелось бы. Предположим, что произошла ошибка при заполнении или выгрузке данных и верным следует считать 1 и 2, соответственно. Заодно сразу же категоризируем данные о количестве детей в семье заёмщиков на 3 группы: нет детей, один ребёнок, два и более детей в семье.

In [18]:
def correct_children(row):
    if row['children'] == 0:
        return 'нет детей'
    if row['children'] == 1:
        return 'один ребёнок'
    if row['children'] == -1:
        return 'один ребёнок'
    if row['children'] == 2:
        return 'два и более'
    if row['children'] == 20:
        return 'два и более'
    if row['children'] == 3:
        return 'два и более'
    if row['children'] == 4:
        return 'два и более'
    if row['children'] == 5:
        return 'два и более'
    
credit_data['children_cor'] = credit_data.apply(correct_children, axis = 1)

In [19]:
#Определим необходиме столбцы для финального анализа
credit_data_fin = credit_data[['children_cor', 'family_status_id', 'category_lvl_income', 'lem_code', 'debt']]

#Разделим итоговую таблицу на 2 части по признаку наличия/отсутствия задолженности
credit_data_fin_debtor = credit_data_fin[credit_data_fin['debt'] == 1]
credit_data_fin_trustworthy = credit_data_fin[credit_data_fin['debt'] == 0]

#А так же проверим, что в итоговой таблице есть все необходимые данные для выводов
display(credit_data_fin.head())

,children_cor,family_status_id,category_lvl_income,lem_code,debt
0,один ребёнок,0,высокий,недвижимость,False
1,один ребёнок,0,низкий,автомобиль,False
2,нет детей,0,средний,недвижимость,False
3,два и более,0,высокий,образование,False
4,нет детей,1,средний,свадьба,False


**Вывод**

Выделили 2 словаря: с уровнем образования (на всякий случай) и с семейным положением (он поможет считать выводы).
Категоризировали данные с уровнем дохода, подбирая по 1/3 выборки методом credit_data[credit_data['total_income'] </> n] для простоты дальнейших выводов в части влияния уровня дохода на возврат кредита в срок.
На этом же шаге устранили артефакты в столбце 'children', заменив значения на предположительно корректные.
В итоговую таблицу добавили необходимые для анализа столбцы, а так же разделили её на 2 части по признаку наличия/отсутствия задолженности, чтобы на следующем шаге сравнивать сводные показатели первой и второй таблиц.

## Шаг 3. Ответьте на вопросы
<a id="answer_the_questions"></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [20]:
#Определим общую долю людей с просрочкой по кредиту
debtor_ratio = len(credit_data_fin_debtor) / len(credit_data_fin)
print('Доля заёмщиков, имевших задолженность по возврату кредитов составляет {:.2%}'.format(debtor_ratio))

print()

#Выведем на информацию для анализа зависимости наличия детей и своевременным возвратом кредита
print('Среди благонадёжных заёмщиков нет детей у {:.2%}'.format(len(credit_data_fin_trustworthy[credit_data_fin_trustworthy['children_cor'] == 'нет детей']) / len(credit_data_fin_trustworthy)), 'выборки, а среди должников – {:.2%}'.format(len(credit_data_fin_debtor[credit_data_fin_debtor['children_cor'] == 'нет детей']) / len(credit_data_fin_debtor)))
print('Среди благонадёжных заёмщиков один ребёнок у {:.2%}'.format(len(credit_data_fin_trustworthy[credit_data_fin_trustworthy['children_cor'] == 'один ребёнок']) / len(credit_data_fin_trustworthy)), 'выборки, а среди должников – {:.2%}'.format(len(credit_data_fin_debtor[credit_data_fin_debtor['children_cor'] == 'один ребёнок']) / len(credit_data_fin_debtor)))
print('Среди благонадёжных заёмщиков 2 и более детей у {:.2%}'.format(len(credit_data_fin_trustworthy[credit_data_fin_trustworthy['children_cor'] == 'два и более']) / len(credit_data_fin_trustworthy)), 'выборки, а среди должников – {:.2%}'.format(len(credit_data_fin_debtor[credit_data_fin_debtor['children_cor'] == 'два и более']) / len(credit_data_fin_debtor)))


Доля заёмщиков, имевших задолженность по возврату кредитов составляет 8.09%

Среди благонадёжных заёмщиков нет детей у 66.14% выборки, а среди должников – 61.06%
Среди благонадёжных заёмщиков один ребёнок у 22.34% выборки, а среди должников – 25.56%
Среди благонадёжных заёмщиков 2 и более детей у 11.51% выборки, а среди должников – 13.38%


**Вывод**

Заёмщики без детей с бОльшей вероятностью вернут кредит вовремя, соответственно, наличие детей увеличивает вероятность просочки по кредиту.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [21]:
print('Среди благонадёжных заёмщиков семейное положение женат / замужем у {:.2%}'.format(len(credit_data_fin_trustworthy[credit_data_fin_trustworthy['family_status_id'] == 0]) / len(credit_data_fin_trustworthy)), ', среди должников – {:.2%}'.format(len(credit_data_fin_debtor[credit_data_fin_debtor['family_status_id'] == 0]) / len(credit_data_fin_debtor)))
print('Среди благонадёжных заёмщиков семейное положение в гражданском браке у {:.2%}'.format(len(credit_data_fin_trustworthy[credit_data_fin_trustworthy['family_status_id'] == 1]) / len(credit_data_fin_trustworthy)), ', среди должников – {:.2%}'.format(len(credit_data_fin_debtor[credit_data_fin_debtor['family_status_id'] == 1]) / len(credit_data_fin_debtor)))
print('Среди благонадёжных заёмщиков семейное положение вдовец / вдова у {:.2%}'.format(len(credit_data_fin_trustworthy[credit_data_fin_trustworthy['family_status_id'] == 2]) / len(credit_data_fin_trustworthy)), ', среди должников – {:.2%}'.format(len(credit_data_fin_debtor[credit_data_fin_debtor['family_status_id'] == 2]) / len(credit_data_fin_debtor)))
print('Среди благонадёжных заёмщиков семейное положение в разводе у {:.2%}'.format(len(credit_data_fin_trustworthy[credit_data_fin_trustworthy['family_status_id'] == 3]) / len(credit_data_fin_trustworthy)), ', среди должников – {:.2%}'.format(len(credit_data_fin_debtor[credit_data_fin_debtor['family_status_id'] == 3]) / len(credit_data_fin_debtor)))
print('Среди благонадёжных заёмщиков семейное положение не женат / не замужем у {:.2%}'.format(len(credit_data_fin_trustworthy[credit_data_fin_trustworthy['family_status_id'] == 4]) / len(credit_data_fin_trustworthy)), ', среди должников – {:.2%}'.format(len(credit_data_fin_debtor[credit_data_fin_debtor['family_status_id'] == 4]) / len(credit_data_fin_debtor)))


Среди благонадёжных заёмщиков семейное положение женат / замужем у 57.87% , среди должников – 53.48%
Среди благонадёжных заёмщиков семейное положение в гражданском браке у 19.15% , среди должников – 22.29%
Среди благонадёжных заёмщиков семейное положение вдовец / вдова у 4.53% , среди должников – 3.62%
Среди благонадёжных заёмщиков семейное положение в разводе у 5.61% , среди должников – 4.88%
Среди благонадёжных заёмщиков семейное положение не женат / не замужем у 12.83% , среди должников – 15.74%


**Вывод**

Женатые / замужние чеще своевременно возвращают средства, чем имеют просрочку. Среди живущих в гражданском браке больше должников, чем благонадёжных заёмщиков. В случае с вдовцами / вдовами и людьми в разводе не наблюдается зависимости. Среди не женатых / не замужних больше вероятность задолженности платежей.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [22]:
print('Без просрочки и с низким уровнем дохода {:.2%}'.format(len(credit_data_fin_trustworthy[credit_data_fin_trustworthy['category_lvl_income'] == 'низкий']) / len(credit_data_fin_trustworthy)), 'с просрочкой – {:.2%}'.format(len(credit_data_fin_debtor[credit_data_fin_debtor['category_lvl_income'] == 'низкий']) / len(credit_data_fin_debtor)))
print('Без просрочки и с средним уровнем дохода {:.2%}'.format(len(credit_data_fin_trustworthy[credit_data_fin_trustworthy['category_lvl_income'] == 'средний']) / len(credit_data_fin_trustworthy)), 'с просрочкой – {:.2%}'.format(len(credit_data_fin_debtor[credit_data_fin_debtor['category_lvl_income'] == 'средний']) / len(credit_data_fin_debtor)))
print('Без просрочки и с высоким уровнем дохода {:.2%}'.format(len(credit_data_fin_trustworthy[credit_data_fin_trustworthy['category_lvl_income'] == 'высокий']) / len(credit_data_fin_trustworthy)), 'с просрочкой – {:.2%}'.format(len(credit_data_fin_debtor[credit_data_fin_debtor['category_lvl_income'] == 'высокий']) / len(credit_data_fin_debtor)))


Без просрочки и с низким уровнем дохода 34.17% с просрочкой – 34.92%
Без просрочки и с средним уровнем дохода 32.12% с просрочкой – 33.66%
Без просрочки и с высоким уровнем дохода 33.71% с просрочкой – 31.42%


**Вывод**

Уровень дохода не влияет на своевременный возврат кредита.

- Как разные цели кредита влияют на его возврат в срок?

In [23]:
print('На свадьбу без просрочки {:.2%}'.format(len(credit_data_fin_trustworthy[credit_data_fin_trustworthy['lem_code'] == 'свадьба']) / len(credit_data_fin_trustworthy)), 'с просрочкой – {:.2%}'.format(len(credit_data_fin_debtor[credit_data_fin_debtor['lem_code'] == 'свадьба']) / len(credit_data_fin_debtor)))
print('На жильё без просрочки {:.2%}'.format(len(credit_data_fin_trustworthy[credit_data_fin_trustworthy['lem_code'] == 'недвижимость']) / len(credit_data_fin_trustworthy)), 'с просрочкой – {:.2%}'.format(len(credit_data_fin_debtor[credit_data_fin_debtor['lem_code'] == 'недвижимость']) / len(credit_data_fin_debtor)))
print('На авто без просрочки {:.2%}'.format(len(credit_data_fin_trustworthy[credit_data_fin_trustworthy['lem_code'] == 'автомобиль']) / len(credit_data_fin_trustworthy)), 'с просрочкой – {:.2%}'.format(len(credit_data_fin_debtor[credit_data_fin_debtor['lem_code'] == 'автомобиль']) / len(credit_data_fin_debtor)))
print('На образование без просрочки {:.2%}'.format(len(credit_data_fin_trustworthy[credit_data_fin_trustworthy['lem_code'] == 'образование']) / len(credit_data_fin_trustworthy)), 'с просрочкой – {:.2%}'.format(len(credit_data_fin_debtor[credit_data_fin_debtor['lem_code'] == 'образование']) / len(credit_data_fin_debtor)))


На свадьбу без просрочки 10.93% с просрочкой – 10.68%
На жильё без просрочки 50.84% с просрочкой – 44.92%
На авто без просрочки 19.77% с просрочкой – 23.15%
На образование без просрочки 18.46% с просрочкой – 21.25%


**Вывод**

Кредиты на недвижимость чаще платят вовремя, а по платежам за автомобиль и образование имеют задолженности чуть чаще, но явной зависимости возврата кредита в срок от цели не обнаружено.

## Шаг 4. Общий вывод
<a id="recap"></a>

Для исследования мы получили таблицу с данными о клиентах банка. Мы знали: стаж, уровень дохода, пол и возраст, количество детей в семье, тип занятости клиента, уровень образования, цель получения кредита и факт наличия задолженности по возврату кредита. Часть данных была пропущенна или некорректна: *при очередных выгрузках из того же источника советуем сразу же обратить внимание на столбцы со стажем и уровнем дохода.*

На предварительном этапе мы:
- заполнили пропуски в данных;
- нашли несколько артефактов и (чувствительные для результатов исследования значения в столбце с количеством детей скорректировали, пол 'XNA' игнорировали, т.к. на выводы это не повлияет);
- заменили типы данных, чем сэкономили 66% используемой датафреймом памяти;
- удалили дубликаты, чтобы не потерять в качестве выводов;
- лемматизировали цели получения кредитов;
- категоризировали выборку по цели получения кредита, уровню дохода и количеству детей в семье.

Интересно, что 8% заёмщиков когда-либо имели задолженность по возврату кредита. Влияние различных факторов на наличие задолженности рапределилось следующим образом:
1. **Наличие детей.** Среди клиентов без задолженности по возврату кредитов не имеют детей 66%, один ребёнок у 22%, два и более у 12% клиентов, тогда как среди клиентов, имеющих задолженность не имеют детей 61%, один ребёнок у 26%, два и более детей у 13% клиентов. *Наличие детей повышает вероятность возникновения задолженности по возврату кредитов на 5%.*
2. **Семейное положение.** Доля благонадёжных заёмщиков, состоящих в официальном браке на 4% больше, чем должников с таким же семейным положением. Состоящих в гражданском браке или не женатых клиентов на 3% больше с задолженностью, чем без неё. Разница среди вдовцов и разведённых несущественная. *Официальный брак увеличивает вероятность своевременного возврата кредита.*
3. **Уровень дохода.** Клиентов с низким уровнем дохода по 34% как среди имеющих, так и среди неимеющих задолженность по кредиту. Клиентов со средним уровнем дохода на 1,5% больше среди имеющих задолженность. Тогда как клиентов с высоким уровнем дохода больше среди благонадёжных заёмщиков. Корреляции не наблюдается и разница в процентах невелика. *Уровень дохода не влияет на вероятность возникновения задолженности по возврату кредита.*
4. **Цели получения кредита.** Доля клиентов, получавших кредит на свадьбу по 10% как среди вовремя выплачивавших, так и среди имевших задолженность. Среди тех, кто платил вовремя, на недвижимость кредит брали 51%, а среди имевших задолженность – 45%. Среди должников, на 3% чаще встречаются те, кто брал кредит на автомобиль и на образование. *Риск возникновения задолженности выше у тех заёмщиков, которые получают кредит на покупку автомобиля и на получение образования. Кредиты на недвижимость чаще выплачивают вовремя.*

### Совет
При построении модели кредитного скорринга стоит повышать риск клиентам, которые:
- имеют детей;
- состоят в гражданском браке или не женаты / не замужем;
- берут кредит для покупки автомобиля или получения образования.